In [1]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from torch.optim import Adamax
import wandb
import yaml
import argparse
from tqdm import tqdm
import numpy as np
import os
from set_seed import set_seed
from transformers import AutoTokenizer
import torchmetrics
from torch.optim.lr_scheduler import StepLR

from models import SBERT_base_Model, BERT_base_Model
from datasets import KorSTSDatasets, Collate_fn, bucket_pair_indices, KorSTSDatasets_for_BERT

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
print("training on", device)
model = SBERT_base_Model("klue/roberta-large")
for i, (name,param) in enumerate(model.named_parameters()):
    param.requires_grad = False
    if i == 392:
        break



training on cuda


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

0 bert.embeddings.word_embeddings.weight
1 bert.embeddings.position_embeddings.weight
2 bert.embeddings.token_type_embeddings.weight
3 bert.embeddings.LayerNorm.weight
4 bert.embeddings.LayerNorm.bias
5 bert.encoder.layer.0.attention.self.query.weight
6 bert.encoder.layer.0.attention.self.query.bias
7 bert.encoder.layer.0.attention.self.key.weight
8 bert.encoder.layer.0.attention.self.key.bias
9 bert.encoder.layer.0.attention.self.value.weight
10 bert.encoder.layer.0.attention.self.value.bias
11 bert.encoder.layer.0.attention.output.dense.weight
12 bert.encoder.layer.0.attention.output.dense.bias
13 bert.encoder.layer.0.attention.output.LayerNorm.weight
14 bert.encoder.layer.0.attention.output.LayerNorm.bias
15 bert.encoder.layer.0.intermediate.dense.weight
16 bert.encoder.layer.0.intermediate.dense.bias
17 bert.encoder.layer.0.output.dense.weight
18 bert.encoder.layer.0.output.dense.bias
19 bert.encoder.layer.0.output.LayerNorm.weight
20 bert.encoder.layer.0.output.LayerNorm.bias
21 b

In [3]:
model = SBERT_with_KLUE_BERT()

epochs = 1
criterion = nn.MSELoss()
optimizer = Adam(params=model.parameters(), lr=.2e-5)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
for name, param in model.named_parameters():
    print(name, param.shape)

bert.embeddings.word_embeddings.weight torch.Size([32000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [5]:
train_loss = []
valid_loss = []

for epoch in range(epochs):
    for data in train_loader:
        s1, s2, label = data
        logits = model(s1, s2)
        loss = criterion(logits, label)
        train_loss.append(loss.detach())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    val_loss = 0
    with torch.no_grad():
        for i, data in enumerate(valid_loader):
            s1, s2, label = data
            logits = model(s1, s2)
            loss = criterion(logits, label)
            val_loss += loss
    valid_loss.append(val_loss.detach()/i)


KeyboardInterrupt: 

In [6]:
train_loss

[tensor(21.4045),
 tensor(5.1395),
 tensor(5.2421),
 tensor(7.4910),
 tensor(7.2007),
 tensor(7.2855),
 tensor(4.5699),
 tensor(3.8467)]